In [2]:
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import joblib  # Pour sauvegarder et recharger le modèle

# Charger les données
data = pd.read_csv("./datatest/DatasetmalwareExtrait.csv")

# Séparer les caractéristiques (X) et la cible (y)
X = data.drop(columns=['legitimate'])
y = data['legitimate']

# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

# Fonction d'optimisation Optuna
def objective(trial):
    # Définir les hyperparamètres à optimiser
    n_estimators = trial.suggest_int('n_estimators', 50, 150)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depth = trial.suggest_int('max_depth', 5, 50, step=5)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])

    # Initialiser le modèle avec ces hyperparamètres
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42,
        n_jobs=-1
    )

    # Utiliser la validation croisée pour évaluer le modèle
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)

    # Retourner la moyenne des scores comme objectif
    return score.mean()

# Créer une étude Optuna
study = optuna.create_study(direction='maximize')

# Lancer l'optimisation
study.optimize(objective, n_trials=30)

# Afficher les meilleurs hyperparamètres
print("Meilleurs hyperparamètres trouvés :", study.best_params)

# Créer un modèle avec les meilleurs hyperparamètres
best_opt_params = study.best_params
optimized_rf = RandomForestClassifier(**best_opt_params, random_state=42, n_jobs=-1)

# Entraîner le modèle avec les données d'entraînement
optimized_rf.fit(X_train, y_train)

# Sauvegarder le modèle optimisé
joblib.dump(optimized_rf, "optimized_model.pkl")
print("Modèle optimisé sauvegardé sous 'optimized_model.pkl'.")

# Charger le modèle sauvegardé (validation)
loaded_model = joblib.load("optimized_model.pkl")
print("Modèle chargé avec succès.")
# 
# # Faire des prédictions avec le modèle chargé
# y_pred_rf = loaded_model.predict(X_test)
# 
# # Évaluer le modèle
# accuracy = accuracy_score(y_test, y_pred_rf)
# print(f"Précision sur l'ensemble de test : {accuracy:.2f}")
# 
# # Matrice de confusion
# cm = confusion_matrix(y_test, y_pred_rf)
# 
# # Visualiser la matrice de confusion
# plt.figure(figsize=(8, 6))
# sns.heatmap(
#     cm, annot=True, fmt='d', cmap='PiYG',
#     xticklabels=['Non-Malware', 'Malware'],
#     yticklabels=['Non-Malware', 'Malware']
# )
# plt.xlabel('Prédictions')
# plt.ylabel('Vérités')
# plt.title('Matrice de Confusion - Random Forest')
# plt.show()
# 
# # Afficher le rapport de classification
# print("\nRapport de classification :")
# print(classification_report(y_test, y_pred_rf))


[I 2024-12-22 12:28:55,499] A new study created in memory with name: no-name-99345548-83dc-4645-9961-2119acdc2ab2
[I 2024-12-22 12:29:01,028] Trial 0 finished with value: 0.9755616559271683 and parameters: {'n_estimators': 93, 'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 16, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 0 with value: 0.9755616559271683.
[I 2024-12-22 12:29:07,627] Trial 1 finished with value: 0.9878818751353459 and parameters: {'n_estimators': 102, 'criterion': 'gini', 'max_depth': 30, 'min_samples_split': 8, 'min_samples_leaf': 6, 'max_features': 'log2'}. Best is trial 1 with value: 0.9878818751353459.
[I 2024-12-22 12:29:15,216] Trial 2 finished with value: 0.9896765540243374 and parameters: {'n_estimators': 135, 'criterion': 'gini', 'max_depth': 30, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 2 with value: 0.9896765540243374.
[I 2024-12-22 12:29:19,346] Trial 3 finished with value: 0.986806686054

Meilleurs hyperparamètres trouvés : {'n_estimators': 108, 'criterion': 'gini', 'max_depth': 35, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Modèle optimisé sauvegardé sous 'optimized_model.pkl'.
Modèle chargé avec succès.


In [3]:
import pefile

def extract_features(executable_path):
    try:
        # Charger le fichier PE
        pe = pefile.PE(executable_path)

        # Extraire les caractéristiques
        features = {
            'AddressOfEntryPoint': pe.OPTIONAL_HEADER.AddressOfEntryPoint,
            'MajorLinkerVersion': pe.OPTIONAL_HEADER.MajorLinkerVersion,
            'MajorImageVersion': pe.OPTIONAL_HEADER.MajorImageVersion,
            'MajorOperatingSystemVersion': pe.OPTIONAL_HEADER.MajorOperatingSystemVersion,
            'DllCharacteristics': pe.OPTIONAL_HEADER.DllCharacteristics,
            'SizeOfStackReserve': pe.OPTIONAL_HEADER.SizeOfStackReserve,
            'NumberOfSections': len(pe.sections),  
            'ResourceSize': get_resource_size(pe)
        }
        
        return features
    except Exception as e:
        print(f"Erreur lors de l'extraction des caractéristiques : {e}")
        return None

def get_resource_size(pe):
    try:
        # Vérifie si le fichier PE a une entrée de ressources
        if hasattr(pe, 'DIRECTORY_ENTRY_RESOURCE') and pe.DIRECTORY_ENTRY_RESOURCE.entries:
            total_size = 0

            # Parcourt les entrées de ressources
            for entry in pe.DIRECTORY_ENTRY_RESOURCE.entries:
                if hasattr(entry, 'directory') and entry.directory.entries:
                    for sub_entry in entry.directory.entries:
                        if hasattr(sub_entry, 'data') and hasattr(sub_entry.data, 'struct'):
                            total_size += sub_entry.data.struct.Size

            return total_size

        return 0
    except Exception as e:
        # Gère les exceptions de manière sécurisée et affiche une erreur
        print(f"Erreur lors de l'extraction de la taille des ressources : {e}")
        return 0



In [4]:
features =extract_features("./datatest/SteamSetup.exe")
features_df = pd.DataFrame([features])
# Charger le modèle
model = joblib.load("optimized_model.pkl")
prediction = model.predict(features_df)[0]
if prediction == 1:
    print("Le fichier est un Malware.")
else:
    print("Le fichier n'est pas un Malware.")


Le fichier n'est pas un Malware.
